# Operações Básicas com GeoPandas

GeoPandas é como uma mesa de trabalho digital para cartógrafos. Se no ArcGIS ou QGIS você usa menus e botões, aqui vamos fazer as mesmas operações com código - é como aprender os atalhos do teclado para trabalhar mais rápido!

## Objetivos
- Entender o GeoDataFrame (nossa planilha espacial)
- Fazer operações básicas de leitura e escrita (abrir e salvar mapas)
- Manipular dados espaciais (editar nossos mapas)
- Criar visualizações básicas (fazer layouts)

## 1. Configuração do Ambiente

Primeiro, vamos preparar nossa mesa de trabalho digital:

In [ ]:
# Nossas ferramentas digitais:
import geopandas as gpd          # Nossa mesa de trabalho espacial
import pandas as pd              # Nossa planilha de atributos
import matplotlib.pyplot as plt   # Nossa ferramenta de layout
from shapely.geometry import Point, Polygon  # Nossas ferramentas de desenho
import numpy as np               # Nossa calculadora avançada

# Configurando o tamanho padrão dos mapas
# Como ajustar o tamanho do papel
plt.rcParams['figure.figsize'] = (12, 8)

## 2. GeoDataFrame: Conceitos Básicos

Um GeoDataFrame é como uma planilha do Excel turbinada com geometrias. Cada linha é um feature (ponto, linha ou polígono) e cada coluna é um atributo, exatamente como na tabela de atributos do seu SIG favorito!

In [ ]:
# Vamos criar um GeoDataFrame com as capitais do Sudeste
# Como criar uma nova camada de pontos no seu SIG
capitais = {
    'cidade': ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Vitória'],
    'estado': ['SP', 'RJ', 'MG', 'ES'],
    'populacao': [12325232, 6747815, 2722000, 365855],
    'geometry': [
        Point(-46.6333, -23.5505),  # SP (como um pin no mapa)
        Point(-43.1729, -22.9068),  # RJ
        Point(-43.9378, -19.9208),  # BH
        Point(-40.3182, -20.3155)   # Vitória
    ]
}

# Criando o GeoDataFrame
# Como criar uma nova camada com tabela de atributos
gdf_capitais = gpd.GeoDataFrame(capitais, crs="EPSG:4326")
print("Nossa tabela de atributos das capitais:")
print(gdf_capitais)

## 3. Operações Básicas

### 3.1 Leitura e Escrita de Dados

In [ ]:
# Salvando em diferentes formatos
# Como exportar uma camada para diferentes formatos
gdf_capitais.to_file('../data/processed/capitais.geojson', driver='GeoJSON')
gdf_capitais.to_file('../data/processed/capitais.shp')

# Lendo os dados salvos
# Como importar uma camada no seu SIG
capitais_geojson = gpd.read_file('../data/processed/capitais.geojson')
print("\nDados lidos do GeoJSON (como abrir uma camada):")
print(capitais_geojson.head())

### 3.2 Filtragem e Seleção

In [ ]:
# Filtrando por atributos
# Como usar 'Select by Attributes' no seu SIG
grandes_cidades = gdf_capitais[gdf_capitais['populacao'] > 1000000]
print("Cidades com mais de 1 milhão de habitantes:")
print(grandes_cidades[['cidade', 'populacao']])

# Selecionando por localização
# Como usar 'Select by Location' no seu SIG
bbox = [-47, -24, -43, -22]  # [minx, miny, maxx, maxy]
cidades_bbox = gdf_capitais.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]
print("\nCidades dentro da nossa área de interesse:")
print(cidades_bbox['cidade'])

### 3.3 Atributos Espaciais

In [ ]:
# Calculando centroides
# Como usar a ferramenta 'Feature to Point'
print("Coordenadas do centro de cada ponto:")
print(gdf_capitais.geometry.centroid)

# Calculando envelope (bbox)
# Como usar 'Minimum Bounding Geometry'
print("\nRetângulo envolvente de todas as capitais:")
print(gdf_capitais.total_bounds)

### 3.4 Operações com Coordenadas

In [ ]:
# Extraindo coordenadas
# Como ver as coordenadas de cada ponto
gdf_capitais['longitude'] = gdf_capitais.geometry.x
gdf_capitais['latitude'] = gdf_capitais.geometry.y

# Reprojetando para UTM
# Como usar 'Project' no seu SIG
gdf_capitais_utm = gdf_capitais.to_crs('EPSG:32723')  # UTM 23S

print("Coordenadas em graus (como no GPS):")
print(gdf_capitais[['cidade', 'longitude', 'latitude']])
print("\nCoordenadas em metros (como na topografia):")
print(gdf_capitais_utm.geometry)

## 4. Visualização Básica

In [ ]:
# Criando diferentes visualizações
# Como fazer diferentes layouts no seu SIG
fig, ax = plt.subplots(1, 2, figsize=(15, 6))

# Mapa de pontos simples
# Como simbolizar pontos com o mesmo símbolo
gdf_capitais.plot(ax=ax[0], color='red', markersize=50)
ax[0].set_title('Capitais do Sudeste')

# Mapa com símbolos proporcionais
# Como fazer símbolos proporcionais à população
gdf_capitais.plot(ax=ax[1], 
                  markersize=gdf_capitais['populacao']/100000,  # Tamanho proporcional
                  color='blue')
ax[1].set_title('Capitais - Tamanho proporcional à população')

plt.tight_layout()
plt.show()

## 5. Operações com Tabelas

In [ ]:
# Criando dados adicionais
# Como adicionar uma nova tabela com dados
dados_extra = pd.DataFrame({
    'cidade': ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Vitória'],
    'pib': [700000, 350000, 90000, 40000],  # PIB em milhões
    'area': [1521, 1200, 331, 98]  # Área em km²
})

# Juntando os dados
# Como fazer um 'Join' no seu SIG
gdf_completo = gdf_capitais.merge(dados_extra, on='cidade')

# Calculando densidade populacional
# Como usar 'Field Calculator'
gdf_completo['densidade'] = gdf_completo['populacao'] / gdf_completo['area']

print("Dados completos das capitais:")
print(gdf_completo[['cidade', 'populacao', 'area', 'densidade', 'pib']])

## 6. Exercícios Práticos

Vamos praticar como um analista GIS:

1. Manipulação de Dados:
   - Crie uma camada com as 10 maiores cidades do seu estado
   - Adicione dados de população e área
   - Calcule a densidade populacional
   - Faça um mapa temático bonito

2. Análise Espacial Básica:
   - Calcule as distâncias entre as capitais
   - Encontre as duas mais próximas
   - Crie buffers de 100km (como áreas de influência)
   - Visualize os resultados

3. Integração de Dados:
   - Baixe dados extras (PIB, IDH, etc.)
   - Junte com seus dados espaciais
   - Crie mapas temáticos
   - Analise as relações entre as variáveis